In [25]:
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re

### Connecting with Twitter API

In [26]:
%run ./keys.ipynb
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

### Political Sentiment:
- Dataframes of tweets of each party. 
- Dataframes of mentions of users for each party. 

- Machine learning for evalutating the sentiment of tweets and mentions.

## General Functions

In [28]:
def extracting_tweets(account):
    """
    This function extract a data frame with the tweets of an account
    and filter the data frame with the columns selected.
    """
    # extracting the tweets
    tweets = [tweet for tweet in tweepy.Cursor(api.user_timeline,
                                           screen_name=account,
                                           tweet_mode='extended',
                                           exclude_replies=True,
                                              include_rts=False).items(20)]
    # creating the data frame
    tweets_json = [tweet._json for tweet in tweets]
    df_tweets = pd.json_normalize(tweets_json)
    
    # selecting useful columns
    columns_selected = ['user.name', 'created_at', 'id', 'full_text', 'display_text_range', 
                    'source', 'retweet_count', 'favorite_count', 'user.followers_count', 
                    'user.friends_count', 'user.statuses_count']
    
    df_tweets_filtered = df_tweets[columns_selected]
    
    # cleaning date time
    df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])
    
    # cleaning source of the tweet
    list_sources = list(df_tweets_filtered['source'])
    df_tweets_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]
    
    return df_tweets_filtered

In [29]:
def adding_party_column(df, string):
    df['partido'] = string
    return df

In [30]:
def adding_type_post_column(df, string):
    """
    string = publicación o mención
    """
    df['tipo de post'] = string
    return df

In [31]:
def extract_hashtags(column):
    """
    Input: column with all tweets. ['full_text'] in this case.
    Output: a list with used hashtags.
    """
    list_hashtags = list(column)
    hashtags = [re.findall(r"#(\w+)", tweet) for tweet in list_hashtags]
    
    return hashtags

In [32]:
def adding_campaign_column(df, string):
    """
    Input: string = General or Madrid.
    Output: a column with the name of the person.
    """
    df['campaña'] = string
    return df

In [41]:
def madrid_campaign(df, column):
    """
    Input: full_text column.
    Output: a column with if the tweets have Madrid or not.
    """
    for tweet in column:
        if "Madrid" in tweet:
            df['campaña'] == "Madrid"
        else:
            df['campaña'] == "General"
    return df

In [42]:
madrid_campaign(df_pp, df_pp['full_text'])

KeyError: 'campaña'

## 1. PARTIDO POPULAR

In [33]:
df_pp = extracting_tweets('populares')

<ipython-input-28-bb3e154e24e8>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])
<ipython-input-28-bb3e154e24e8>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]


In [34]:
df_pp

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count
0,Partido Popular,2021-04-27 18:59:45+00:00,1387119293884665859,📺 @IdiazAyuso: “Estas elecciones han trascendi...,"[0, 236]",[Twitter Media Studio - LiveCut],20,42,822008,5291,102737
1,Partido Popular,2021-04-27 18:56:12+00:00,1387118400086941699,📺 @IdiazAyuso: “Estamos en un momento decisivo...,"[0, 273]",[Twitter Media Studio - LiveCut],31,65,822008,5291,102737
2,Partido Popular,2021-04-27 18:48:49+00:00,1387116541372682240,📺 @IdiazAyuso a Pablo Iglesias: “Los líderes a...,"[0, 277]",[Twitter Media Studio - LiveCut],39,83,822008,5291,102737
3,Partido Popular,2021-04-27 18:37:17+00:00,1387113640801734657,🔵 @Tonicanto1: “El socialismo decía que se os ...,"[0, 213]",[Twitter Media Studio - LiveCut],29,63,822008,5291,102737
4,Partido Popular,2021-04-27 18:34:55+00:00,1387113042031304708,🔵 @Tonicanto1: “Reflexionad conmigo ¿Qué prefe...,"[0, 268]",[Twitter Media Studio - LiveCut],60,132,822008,5291,102737
5,Partido Popular,2021-04-27 18:26:55+00:00,1387111029956546560,"🔵 @Tonicanto1: “Tic, tac, tic, tac…siete días ...","[0, 278]",[Twitter Media Studio - LiveCut],41,91,822008,5291,102737
6,Partido Popular,2021-04-27 18:19:00+00:00,1387109037813215233,💬 @MariMarBlanco_: “Sánchez tiene los días con...,"[0, 270]",[Twitter Media Studio - LiveCut],28,71,822008,5291,102737
7,Partido Popular,2021-04-27 18:13:54+00:00,1387107754540687367,💬 @MariMarBlanco_: “Tenemos que darlo todo en ...,"[0, 243]",[Twitter Media Studio - LiveCut],24,46,822008,5291,102737
8,Partido Popular,2021-04-27 17:46:01+00:00,1387100738292224008,📡 #EnDirecto 👉 Intervención de @IdiazAyuso en ...,"[0, 127]",[Twitter Media Studio],37,85,822008,5291,102737
9,Partido Popular,2021-04-27 17:10:00+00:00,1387091672530440192,"🔵 A las 19:30h., @IdiazAyuso estará en #Legané...","[0, 184]",[Twitter Web App],18,80,822008,5291,102737


In [ ]:
adding_party_column(df_pp, 'Partido Popular')

In [15]:
adding_type_post_column(df_pp, 'publicación')

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,partido,tipo de post
0,Partido Popular,2021-04-26 17:22:35+00:00,1386732451196776459,📡 #EnDirecto 👉 @IdiazAyuso interviene en un ac...,"[0, 133]",[Twitter Media Studio],19,48,821857,5290,102681,Partido Popular,publicación
1,Partido Popular,2021-04-26 16:40:00+00:00,1386721734884270082,"🔵 En unos minutos, a partir de las 19:00h., @I...","[0, 204]",[Twitter Web App],33,80,821857,5290,102681,Partido Popular,publicación
2,Partido Popular,2021-04-26 15:57:38+00:00,1386711074301435909,🔴 Quien acerca semana sí y semana también a cr...,"[0, 277]",[Twitter Web App],117,249,821857,5290,102681,Partido Popular,publicación
3,Partido Popular,2021-04-26 14:00:26+00:00,1386681579326554117,"""La obsesión del Gobierno por Madrid la pagamo...","[0, 85]",[Twitter Web App],123,286,821857,5290,102681,Partido Popular,publicación
4,Partido Popular,2021-04-26 12:54:59+00:00,1386665109716717571,👉 @pablocasado_ finaliza su intervención con u...,"[0, 163]",[Twitter Media Studio - LiveCut],44,80,821857,5290,102681,Partido Popular,publicación
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Partido Popular,2021-04-20 06:01:41+00:00,1384386772805763075,"☕️ Buenos días y #FelizMartes a todos, aquí os...","[0, 116]",[Twitter for iPhone],12,34,821857,5290,102681,Partido Popular,publicación
196,Partido Popular,2021-04-19 18:03:12+00:00,1384205956582494208,👉 @IdiazAyuso: “Vamos a salir adelante con poc...,"[0, 255]",[Twitter Media Studio - LiveCut],116,279,821857,5290,102681,Partido Popular,publicación
197,Partido Popular,2021-04-19 17:57:04+00:00,1384204414949363723,👉 El Gobierno siempre ha criticado las propues...,"[0, 228]",[Twitter Media Studio - LiveCut],86,166,821857,5290,102681,Partido Popular,publicación
198,Partido Popular,2021-04-19 17:48:30+00:00,1384202257906823173,"👉 @IdiazAyuso: “Nos intentan decir que, como s...","[0, 262]",[Twitter Media Studio - LiveCut],159,447,821857,5290,102681,Partido Popular,publicación


In [23]:
df_pp['hashtags'] = extract_hashtags(df_pp['full_text'])

## 2. PSOE

In [12]:
df_psoe = extracting_tweets('psoe')

<ipython-input-3-9cf1f1161bbc>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])
<ipython-input-3-9cf1f1161bbc>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]


In [13]:
adding_party_column(df_psoe, 'PSOE')
adding_type_post_column(df_psoe, 'publicación')
df_psoe['hashtags'] = extract_hashtags(df_psoe['full_text'])

## 3. PODEMOS

In [14]:
df_podemos = extracting_tweets('PODEMOS')

<ipython-input-3-9cf1f1161bbc>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])
<ipython-input-3-9cf1f1161bbc>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]


In [15]:
adding_party_column(df_podemos, 'PODEMOS')
adding_type_post_column(df_podemos, 'publicación')
df_podemos['hashtags'] = extract_hashtags(df_podemos['full_text'])

## 4. CUIDADANOS

In [16]:
df_ciudadanos = extracting_tweets('ciudadanoscs')

<ipython-input-3-9cf1f1161bbc>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])
<ipython-input-3-9cf1f1161bbc>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]


In [17]:
adding_party_column(df_ciudadanos, 'CIUDADANOS')
adding_type_post_column(df_ciudadanos, 'publicación')
df_ciudadanos['hashtags'] = extract_hashtags(df_ciudadanos['full_text'])

## 5. VOX

In [18]:
df_vox = extracting_tweets('vox_es')

<ipython-input-3-9cf1f1161bbc>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])
<ipython-input-3-9cf1f1161bbc>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]


In [19]:
adding_party_column(df_vox, 'PSOE')
adding_type_post_column(df_vox, 'publicación')
df_vox['hashtags'] = extract_hashtags(df_vox['full_text'])

## All together

In [20]:
frames = [df_pp, df_psoe, df_ciudadanos, df_podemos, df_vox ]
df_final_tweets = pd.concat(frames)

In [21]:
df_final_tweets.shape

(6177, 14)

## Exporting the final dataframe

In [22]:
df_final_tweets.to_csv('../data/processed/df_final_2000.csv')

In [21]:
# TWEETS OF PARTIES -> GENERAL

# Partido Popular (General)
df_pp = extracting_tweets('populares')
adding_party_column(df_pp, 'Partido Popular')
adding_type_post_column(df_pp, 'publicación')
df_pp['hashtags'] = extract_hashtags(df_pp['full_text'])
adding_campaign_column(df_pp, 'general')

# PSOE (General)
df_psoe = extracting_tweets('psoe')
adding_party_column(df_psoe, 'PSOE')
adding_type_post_column(df_psoe, 'publicación')
df_psoe['hashtags'] = extract_hashtags(df_psoe['full_text'])
adding_campaign_column(df_psoe, 'general')

# PODEMOS (General)
df_podemos = extracting_tweets('PODEMOS')
adding_party_column(df_podemos, 'PODEMOS')
adding_type_post_column(df_podemos, 'publicación')
df_podemos['hashtags'] = extract_hashtags(df_podemos['full_text'])
adding_campaign_column(df_podemos, 'general')

# CIUDADANOS (General)
df_ciudadanos = extracting_tweets('ciudadanoscs')
adding_party_column(df_ciudadanos, 'CIUDADANOS')
adding_type_post_column(df_ciudadanos, 'publicación')
df_ciudadanos['hashtags'] = extract_hashtags(df_ciudadanos['full_text'])
adding_campaign_column(df_ciudadanos, 'general')

# VOX (General)
df_vox = extracting_tweets('vox_es')
adding_party_column(df_vox, 'VOX')
adding_type_post_column(df_vox, 'publicación')
df_vox['hashtags'] = extract_hashtags(df_vox['full_text'])
adding_campaign_column(df_vox, 'general')

# TWEETS OF PARTIES -> MADRID

# Partido Popular (Madrid)
df_pp_madrid = extracting_tweets('ppmadrid')
adding_party_column(df_pp_madrid, 'Partido Popular')
adding_type_post_column(df_pp_madrid, 'publicación')
df_pp_madrid['hashtags'] = extract_hashtags(df_pp_madrid['full_text'])
adding_campaign_column(df_pp_madrid, 'madrid')

# PSOE (Madrid)
df_psoe_madrid = extracting_tweets('psoe_m')
adding_party_column(df_psoe_madrid, 'PSOE')
adding_type_post_column(df_psoe_madrid, 'publicación')
df_psoe_madrid['hashtags'] = extract_hashtags(df_psoe_madrid['full_text'])
adding_campaign_column(df_psoe_madrid, 'madrid')

# PODEMOS (Madrid)
df_podemos_madrid = extracting_tweets('podemosmad')
adding_party_column(df_podemos_madrid, 'PODEMOS')
adding_type_post_column(df_podemos_madrid, 'publicación')
df_podemos_madrid['hashtags'] = extract_hashtags(df_podemos_madrid['full_text'])
adding_campaign_column(df_podemos_madrid, 'madrid')

# CIUDADANOS (Madrid)
df_ciudadanos_madrid = extracting_tweets('CsMadridCiudad')
adding_party_column(df_ciudadanos_madrid, 'CIUDADANOS')
adding_type_post_column(df_ciudadanos_madrid, 'publicación')
df_ciudadanos_madrid['hashtags'] = extract_hashtags(df_ciudadanos_madrid['full_text'])
adding_campaign_column(df_ciudadanos_madrid, 'madrid')

# Más Madrid (Madrid)
df_masmad_madrid = extracting_tweets('MasMadridCM')
adding_party_column(df_masmad_madrid, 'Más Madrid')
adding_type_post_column(df_masmad_madrid, 'publicación')
df_masmad_madrid['hashtags'] = extract_hashtags(df_masmad_madrid['full_text'])
adding_campaign_column(df_masmad_madrid, 'madrid')

# VOX (Madrid)
df_vox_madrid = extracting_tweets('VOX_AytoMadrid')
adding_party_column(df_vox_madrid, 'VOX')
adding_type_post_column(df_vox_madrid, 'publicación')
df_vox_madrid['hashtags'] = extract_hashtags(df_vox_madrid['full_text'])
adding_campaign_column(df_vox_madrid, 'madrid')

# TWEETS OF CANDIDATES AND PRESIDENTS

# Pablo Casado (Partido Popular) (General)
df_pcasado = extracting_tweets('pablocasado_')
adding_party_column(df_pcasado, 'Partido Popular')
adding_type_post_column(df_pcasado, 'publicación')
df_pcasado['hashtags'] = extract_hashtags(df_pcasado['full_text'])
adding_campaign_column(df_pcasado, 'general')

# Pedro Sánchez (PSOE) (General)
df_psanchez = extracting_tweets('sanchezcastejon')
adding_party_column(df_psanchez, 'PSOE')
adding_type_post_column(df_psanchez, 'publicación')
df_psanchez['hashtags'] = extract_hashtags(df_psanchez['full_text'])
adding_campaign_column(df_psanchez, 'general')

# Pablo Iglesias (PODEMOS) (General)
df_piglesias = extracting_tweets('PabloIglesias')
adding_party_column(df_piglesias, 'PODEMOS')
adding_type_post_column(df_piglesias, 'publicación')
df_piglesias['hashtags'] = extract_hashtags(df_piglesias['full_text'])
adding_campaign_column(df_piglesias, 'general')

# Inés Arrimadas (CIUDADANOS) (General)
df_iarrimadas = extracting_tweets('InesArrimadas')
adding_party_column(df_iarrimadas, 'CIUDADANOS')
adding_type_post_column(df_iarrimadas, 'publicación')
df_iarrimadas['hashtags'] = extract_hashtags(df_iarrimadas['full_text'])
adding_campaign_column(df_iarrimadas, 'general')

# Santiago Abascal (VOX) (General)
df_abascal = extracting_tweets('Santi_ABASCAL')
adding_party_column(df_abascal, 'VOX')
adding_type_post_column(df_abascal, 'publicación')
df_abascal['hashtags'] = extract_hashtags(df_abascal['full_text'])
adding_campaign_column(df_abascal, 'general')

# Isabel Díaz Ayuso (Partido Popular) (Madrid)
df_ayuso = extracting_tweets('IdiazAyuso')
adding_party_column(df_ayuso, 'Partido Popular')
adding_type_post_column(df_ayuso, 'publicación')
df_ayuso['hashtags'] = extract_hashtags(df_ayuso['full_text'])
adding_campaign_column(df_ayuso, 'madrid')

# Isabel Díaz Ayuso (Partido Popular) (Madrid)
df_ayuso = extracting_tweets('IdiazAyuso')
adding_party_column(df_ayuso, 'CIUDADANOS')
adding_type_post_column(df_ayuso, 'publicación')
df_ayuso['hashtags'] = extract_hashtags(df_ayuso['full_text'])
adding_campaign_column(df_ayuso, 'madrid')

# Ángel Gabilondo (PSOE) (Madrid)
df_gabilondo = extracting_tweets('equipoGabilondo')
adding_party_column(df_gabilondo, 'CIUDADANOS')
adding_type_post_column(df_gabilondo, 'publicación')
df_gabilondo['hashtags'] = extract_hashtags(df_gabilondo['full_text'])
adding_campaign_column(df_gabilondo, 'madrid')

# Rocío Monasterio (VOX) (Madrid)
df_monasterio = extracting_tweets('monasterioR')
adding_party_column(df_monasterio, 'VOX')
adding_type_post_column(df_monasterio, 'publicación')
df_monasterio['hashtags'] = extract_hashtags(df_monasterio['full_text'])
adding_campaign_column(df_monasterio, 'madrid')

# Mónica García (Más Madrid) (Madrid)
df_mgarcia = extracting_tweets('Monica_Garcia_G')
adding_party_column(df_mgarcia, 'Más Madrid')
adding_type_post_column(df_mgarcia, 'publicación')
df_mgarcia['hashtags'] = extract_hashtags(df_mgarcia['full_text'])
adding_campaign_column(df_mgarcia, 'madrid')

# Mónica García (Más Madrid) (Madrid)
df_edmundo = extracting_tweets('BalEdmundo')
adding_party_column(df_edmundo, 'CIUDADANOS')
adding_type_post_column(df_edmundo, 'publicación')
df_edmundo['hashtags'] = extract_hashtags(df_edmundo['full_text'])
adding_campaign_column(df_edmundo, 'madrid')

# ALL TOGETHER
frames = [df_pp, df_psoe, df_ciudadanos, df_podemos, df_vox,
          df_ciudadanos_madrid, df_masmad_madrid, df_pp_madrid, df_psoe_madrid, df_podemos_madrid, df_vox_madrid,
          df_pcasado, df_psanchez, df_piglesias, df_abascal, df_iarrimadas,
          df_ayuso, df_gabilondo, df_mgarcia, df_monasterio, df_edmundo]

df_final_tweets = pd.concat(frames)


df_final_tweets.to_csv('../data/allpartiespeople_try1.csv', index=False)


<ipython-input-20-bb3e154e24e8>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])
<ipython-input-20-bb3e154e24e8>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]


In [50]:
#df_mentions_filtered['created_at'] = pd.to_datetime(df_mentions_filtered['created_at'])
df_pp.loc[:, 'created_at'] = pd.to_datetime(df_pp['created_at'])

In [54]:
list_sources = list(df_pp['source'])

In [55]:
df_pp.loc[:, 'source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]

TypeError: expected string or bytes-like object